In [1]:
import numpy as np
import scipy.io
import pandas
from tools.countries import*
from scipy import linalg
import time

Simulation part:

In [2]:
#####User defined parameters
N = 2
T = 2000
#####Model parameters
gam = 1
alpha = 0.36
beta = 0.99
delta = 0.025
rho = 0.95
sigma = 0.01

######################################
####Variance-covariance matrix
vcv = sigma**(2)*(np.eye(N)+ np.ones(N))
####Normalising constant
A = (1-beta+beta*delta)/alpha/beta
####welfare weight
tau = 1
######################################
#The above normalization ensures that steady state
k = np.ones((T+1,N))
a = np.ones((1,N))
#####################################
#construct productivity levels
a20200 = Productivity(T,N,a,sigma,rho)

GSSA parameters

In [3]:
#checking part with matlab
a20200 = scipy.io.loadmat("aT20200N10.mat").get("a20200")
a20200 = a20200[:T, :N]

In [4]:
#Compute a first-degree polynomial solution using the one-node Monte Carlo integration method (this #solution will be used as an initial guess for the other cases) 

start = time.time()
kdamp = 0.1
dif_1d = 1e+10
# Initialize the first-degree capital policy functions of N countries 
bk_1d= np.vstack((np.zeros((1,N)), np.diag(0.9*np.ones(N)),np.diag(0.1*np.ones(N))))
# Initialize the capital series
k_old = np.ones((T+1,N))
# The main iterative cycle of GSSA
while dif_1d > 1e-4*kdamp:
    #construct starting numpy for value storages
    x = np.empty((T,2*N+1))
    for i in range(T):
        x[i]= np.hstack((1,k[i], a20200[i]))
        k[i+1]= np.hstack((1,k[i], a20200[i]))@bk_1d
        #Compute consumption series 
    C = (A*k[:T]**alpha*a20200[:T]-k[1:T+1]+(1-delta)*k[:T])@np.ones((N,1))
    #Individual consumption is the same for all countries
    c = C@np.ones((1,N))/N

    #Evaluate the percentage (unit-free) difference between the series from the previous and current iterations
    dif_1d = np.mean(abs(1-k/k_old))
    #Monte Carlo realizations of the right side of the Euler equation
    #Consturct containers
    Y = np.empty((T-1,N))
    for i in range(N):
        Y[:T-1,i]= beta*c[1:T,i]**(-gam)/c[:T-1,i]**(-gam)*(1-delta+alpha*A*k[1:T,i]**(alpha-1)*a20200[1:T,i])*k[1:T,i]

    #Compute and update the coefficients of the capital policy functions
    bk_hat_1d = linalg.inv(x[:T-1].conj().T@x[:T-1])@x[:T-1].conj().T@Y[:T-1]
    bk_1d = kdamp*bk_hat_1d+(1-kdamp)*bk_1d
    k_old = k.copy()
end = time.time()
Ncountry_GSSA_1time = end-start

In [5]:
#Compute polynomial solutions of the degrees from one to D_max using one of the following integration methods: Monte Carlo, Gauss-Hermite product and monomial non-product methods


#Damping parameter for (fixed-point) iteration on the coefficients of the capital policy functions
kdamp     = 0.1
#Set the initial difference between the series from two iterations in the convergence criterion   
dif_GSSA_D  = 1e+10

#The matrix of the polynomial coefficients
D_max = 5

#construct container
npol = np.empty((1,D_max))

for i in range(1, D_max+1):
    npol[0,i-1]= Ord_Polynomial_N(np.hstack((k[0],a20200[0]))[np.newaxis,], i).shape[1]